In [88]:
import pandas as pd
import numpy as np
from PIL import Image
import os
from os import path
from pathlib import Path
import sys
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import cv2
import webcolors
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib import colors
from skimage.io import imshow, imread
from skimage.color import rgb2hsv, hsv2rgb
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from IPython.display import clear_output
from IPython.display import display, HTML
sys.path.append(path.abspath('../MSc-CS-Project---ColourPaletteExtractor-master/'))
from colourpaletteextractor.model import model

### Choose directory of images you want to analsye here

In [24]:
foldername = ''
@interact
def choose_img_folder(folder = [f for f in os.listdir('../') if '.' not in f]):
    global foldername
    foldername = folder

interactive(children=(Dropdown(description='folder', options=('joseph_Mallord_william_turner', 'LICENSE', 'MSc…

In [37]:
art_meta = pd.read_csv('../' + foldername + '.csv')
art_colour = pd.read_csv('../' + foldername + '_colours.csv')
# art_meta[art_meta['Accession_number'] == '700']
art_meta

,Title,Artist,Creation_Date,estimate_year,url,img_URL,Medium,height_cm,width_cm,Accession_number
0,Barra,Samuel John Peploe,c.1902–1903,1902.0,https://artuk.org/discover/artworks/barra-8563...,https://d3d00swyhr67nd.cloudfront.net/w250/col...,oil on panel,18.7,24.1,3039
1,Old Tom Morris,Samuel John Peploe,c.1901,1901.0,https://artuk.org/discover/artworks/old-tom-mo...,https://d3d00swyhr67nd.cloudfront.net/w250/col...,oil on canvas,127.0,101.6,3313
2,Roses,Samuel John Peploe,NaN,NaN,https://artuk.org/discover/artworks/roses-5661...,https://d3d00swyhr67nd.cloudfront.net/w250/col...,oil on board,44.0,39.0,MIDMA_FA_1029
3,Roses,Samuel John Peploe,c.1920,1920.0,https://artuk.org/discover/artworks/roses-8563...,https://d3d00swyhr67nd.cloudfront.net/w250/col...,oil on canvas,50.8,40.6,2810
4,Landscape,Samuel John Peploe,NaN,NaN,https://artuk.org/discover/artworks/landscape-...,https://d3d00swyhr67nd.cloudfront.net/w250/col...,oil on canvas,33.0,40.6,2338
...,...,...,...,...,...,...,...,...,...,...
195,Tulips in a Pottery Vase,Samuel John Peploe,c.1912,1912.0,https://artuk.org/discover/artworks/tulips-in-...,https://d3d00swyhr67nd.cloudfront.net/w250/col...,oil on canvas,45.6,40.7,GLAHA_41371
196,Bathers (Étaples),Samuel John Peploe,1906,1906.0,https://artuk.org/discover/artworks/bathers-ta...,NaN,oil on canvas laid on board,18.2,23.8,GLAHA_43999
197,"Clouds and Sky, Iona",Samuel John Peploe,1928,1928.0,https://artuk.org/discover/artworks/clouds-and...,NaN,oil on canvas,40.7,45.7,GLAHA_43996
198,"New Abbey, Dumfriesshire (Summer)",Samuel John Peploe,1926,1926.0,https://artuk.org/discover/artworks/new-abbey-...,NaN,oil on canvas,56.2,76.1,GLAHA_44001


In [38]:
def colour_palette_to_tuple_rgba(row):
    templist = []
    for i in range(3):
        if row[i] < 0:
            templist.append(256 + row[i])
        else:
            templist.append(row[i])
    return (templist[0] / 255, templist[1] / 255, templist[2] / 255)

def colour_palette_to_positive(row):
    templist = []
    for i in range(3):
        if row[i] < 0:
            templist.append(256 + row[i])
        else:
            templist.append(row[i])
    return (templist[0], templist[1], templist[2])

def colour_palette_to_positive_int(row):
    templist = []
    for i in range(3):
        if row[i] < 0:
            templist.append(256 + row[i])
        else:
            templist.append(row[i])
    return (int(templist[0]), int(templist[1]), int(templist[2]))

def show_color_pie(num):
    Tasks = art_colour[art_colour["id"] == num]["freqency"].values.tolist()
    col = art_colour[art_colour["id"] == num][['R', 'G', 'B']].values.tolist()
    my_labels = [colour_palette_to_positive(col[i]) for i in range(len(col))]
    plt.figure(figsize=(30, 30))
    plt.pie(Tasks, labels = my_labels, 
            colors = [colour_palette_to_tuple_rgba(col[i]) for i in range(len(col))],
           autopct = '%1.0f%%',
        textprops={'fontsize': 30}
           )
    plt.title('Colour Palettes', fontsize=50)
    plt.axis('equal')

@interact
def show_img_and_pie(image = [str(n) + ". " for n in range(len(art_meta))] + art_meta["Title"]):
    num = int(image.split('.')[0])
    image = cv2.imread("../" + foldername + "/" + str(art_meta["Accession_number"][num]) + ".jpg")
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(5, 5))
    imshow(image);
    show_color_pie(num)
    plt.show()

interactive(children=(Dropdown(description='image', options=('0. Barra', '1. Old Tom Morris', '2. Roses', '3. …

In [89]:
@interact
def show_img_and_3dRGB(image = [str(n) + ". " for n in range(len(art_meta))] + art_meta["Title"]):
#     clear_output(wait=True)
    num = int(image.split('.')[0])
    image = cv2.imread("../" + foldername + "/" + str(art_meta["Accession_number"][num]) + ".jpg")
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(5, 5))
    imshow(image);
    
    r, g, b = cv2.split(image)
    fig = plt.figure(figsize=(10, 10))
    axis = fig.add_subplot(1, 1, 1, projection="3d")
    pixel_colors = image.reshape((np.shape(image)[0]*np.shape(image)[1], 3))
    norm = colors.Normalize(vmin=-1.,vmax=1.)
    norm.autoscale(pixel_colors)
    pixel_colors = norm(pixel_colors).tolist()
    axis.scatter(r.flatten(), g.flatten(), b.flatten(), facecolors = pixel_colors, marker=".")
    axis.set_xlabel("Red")
    axis.set_ylabel("Green")
    axis.set_zlabel("Blue")
    
    plt.show()

interactive(children=(Dropdown(description='image', options=('0. Barra', '1. Old Tom Morris', '2. Roses', '3. …

In [90]:
def hue_sat_3d(number):
    img = cv2.imread("../" + foldername + "/" + str(art_meta["Accession_number"][number]) + ".jpg")
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    hsv_img = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    h, s, v = cv2.split(hsv_img)
    
    pixel_colors = img.reshape((np.shape(img)[0]*np.shape(img)[1], 3))
    norm = colors.Normalize(vmin=-1.,vmax=1.)
    norm.autoscale(pixel_colors)
    pixel_colors = norm(pixel_colors).tolist()
    
    fig = plt.figure(figsize=(15, 15))
    axis = fig.add_subplot(1, 1, 1, projection="3d")
    axis.scatter(h.flatten(), s.flatten(), v.flatten(), facecolors = pixel_colors, marker=".")
    axis.set_xlabel("Hue")
    axis.set_ylabel("Saturation")
    axis.set_zlabel("Value")
    plt.show()

@interact
def show_img_and_3dHSV(image = [str(n) + ". " for n in range(len(art_meta))] + art_meta["Title"]):
    num = int(image.split('.')[0])
    image = cv2.imread("../" + foldername + "/" + str(art_meta["Accession_number"][num]) + ".jpg")
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(5, 5))
    imshow(image);
    hue_sat_3d(num)
    plt.show()

interactive(children=(Dropdown(description='image', options=('0. Barra', '1. Old Tom Morris', '2. Roses', '3. …

In [92]:
hsv_csv = pd.DataFrame(columns = ['Hue', 'Saturation', 'Value'])
def hue_sat_value_csv(number):
    img = cv2.imread("../" + foldername + "/" + str(art_meta["Accession_number"][number]) + ".jpg")
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    hsv_img = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    h, s, v = cv2.split(hsv_img)
    hsv_csv['Hue'] = pd.Series(h.flatten())
    hsv_csv['Saturation'] = pd.Series(s.flatten())
    hsv_csv['Value'] = pd.Series(v.flatten())

In [93]:
hue_sat_value_csv(155)

In [94]:
np.mean(hsv_csv["Hue"])

36.29344820554649

In [95]:
@interact (r = (0, 255, 1), g = (0, 255,1), b = (0, 255,1))
def filterRGB_img(image = [str(n) + ". " for n in range(len(art_meta))] + art_meta["Title"], 
                    r=0, g=0, b = 0,
                 directory = widgets.Text(value='Rename_File_Here.jpg', disabled=False)):
    num = int(image.split('.')[0])
    img = imread("../" + foldername + "/" + str(art_meta["Accession_number"][num]) + ".jpg")
    img_filtered = (img[:,:,0] >= r) & (img[:,:,1] >= g) & (img[:,:,2] >= b)
    plt.figure(num=None, figsize=(10, 10))
    img_new = img.copy()
    img_new[:, :, 0] = img_new[:, :, 0] * img_filtered
    img_new[:, :, 1] = img_new[:, :, 1] * img_filtered
    img_new[:, :, 2] = img_new[:, :, 2] * img_filtered
    plt.imshow(img_new);
    plt.show()
    
    def clicked(arg):
        download_img = Image.fromarray(img_new, 'RGB')
        download_img.save(directory)
        print("saved", end='\r')
    
    save_button = widgets.Button(description = 'Save image')   
    save_button.on_click(clicked)
    display(save_button)

interactive(children=(Dropdown(description='image', options=('0. Barra', '1. Old Tom Morris', '2. Roses', '3. …

In [96]:
@interact
def img_HSV(image = [str(n) + ". " for n in range(len(art_meta))] + art_meta["Title"],
           option = ["All", 'Hue','Saturation','Value']):
    num = int(image.split('.')[0])
    img = cv2.imread("../" + foldername + "/" + str(art_meta["Accession_number"][num]) + ".jpg")
    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    if option == "All":
        hsv_list = ['Hue','Saturation','Value']
        fig, ax = plt.subplots(1, 3, figsize=(15,7), sharey = True)

        ax[0].imshow(img_hsv[:,:,0], cmap = 'hsv')
        ax[0].set_title(hsv_list[0], fontsize = 20)
        ax[0].axis('off')

        ax[1].imshow(img_hsv[:,:,1], cmap = 'Greys')
        ax[1].set_title(hsv_list[1], fontsize = 20)
        ax[1].axis('off')

        ax[2].imshow(img_hsv[:,:,2], cmap = 'gray')
        ax[2].set_title(hsv_list[2], fontsize = 20)
        ax[2].axis('off')
        
        fig.tight_layout()
        
    elif option == "Hue":
        plt.figure(num=None, figsize=(8, 6))
        plt.imshow(img_hsv[:,:,0], cmap='hsv')
        plt.colorbar()
    
    elif option == "Saturation":
        plt.figure(num=None, figsize=(8, 6))
        plt.imshow(img_hsv[:,:,1], cmap='Greys')
        plt.colorbar()
    
    elif option == "Value":
        plt.figure(num=None, figsize=(8, 6))
        plt.imshow(img_hsv[:,:,2], cmap='gray')
        plt.colorbar()
    
    plt.show()

interactive(children=(Dropdown(description='image', options=('0. Barra', '1. Old Tom Morris', '2. Roses', '3. …

In [97]:
@interact (Saturation = (0, 255, 1))
def img_HSV_filter(image = [str(n) + ". " for n in range(len(art_meta))] + art_meta["Title"],
                    h = widgets.IntRangeSlider(description='Hue', value=[0, 255],
                                               min = 0, max = 255, step=1),
                    Saturation = 0,
                  directory = widgets.Text(value='Rename_File_Here.jpg', disabled=False)):
    
    num = int(image.split('.')[0])
    img = cv2.imread("../" + foldername + "/" + str(art_meta["Accession_number"][num]) + ".jpg")
    image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    
    lower_mask = img_hsv[:,:,0] >= int(h[0])
    upper_mask = img_hsv[:,:,0] <= int(h[1])
    saturation = img_hsv[:,:,1] >= Saturation
    mask = upper_mask * lower_mask * saturation
    red = image[:,:,0] * mask
    green = image[:,:,1] * mask
    blue = image[:,:,2] * mask
    image_masked = np.dstack((red,green,blue))
    plt.figure(num=None, figsize=(15, 15))
    plt.imshow(image_masked);
    
    plt.show()
    
    def clicked(arg):
        download_img = Image.fromarray(image_masked, 'RGB')
        download_img.save(directory)
        print("saved", end='\r')
    
    save_button = widgets.Button(description = 'Save image')   
    save_button.on_click(clicked)
    display(save_button)

interactive(children=(Dropdown(description='image', options=('0. Barra', '1. Old Tom Morris', '2. Roses', '3. …